<a href="https://colab.research.google.com/github/tx1103mark/tweet-sentiment/blob/master/TPUs_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPUs in Colab&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>
In this example, we'll work through training a model to classify images of
flowers on Google's lightning-fast Cloud TPUs. Our model will take as input a photo of a flower and return whether it is a daisy, dandelion, rose, sunflower, or tulip.

We use the Keras framework, new to TPUs in TF 2.1.0. Adapted from [this notebook](https://colab.research.google.com/github/GoogleCloudPlatform/training-data-analyst/blob/master/courses/fast-and-lean-data-science/07_Keras_Flowers_TPU_xception_fine_tuned_best.ipynb) by [Martin Gorner](https://twitter.com/martin_gorner).

#### License

Copyright 2019-2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


---


This is not an official Google product but sample code provided for an educational purpose.


## Enabling and testing the TPU

First, you'll need to enable TPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select TPU from the Hardware Accelerator drop-down

Next, we'll check that we can connect to the TPU:

#Data process

In [ ]:
/**
 * Copyright 2020 Huawei Technologies Co., Ltd
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

#ifndef MINDSPORE_LITE_TOOLS_CONVERTER_PARSER_TF_NODE_PARSER_REGISTRY_H
#define MINDSPORE_LITE_TOOLS_CONVERTER_PARSER_TF_NODE_PARSER_REGISTRY_H

#include <string>
#include <unordered_map>
#include "tf_node_parser.h"

namespace mindspore {
namespace lite {
class TFNodeParserRegistry {
 public:
  TFNodeParserRegistry() = default;

  virtual ~TFNodeParserRegistry();

  static TFNodeParserRegistry *GetInstance();
  TFNodeParser *GetNodeParser(const std::string &name);

  std::unordered_map<std::string, TFNodeParser *> parsers;
};

class TFNodeRegistrar {
 public:
  TFNodeRegistrar(const std::string &name, TFNodeParser *parser) {
    TFNodeParserRegistry::GetInstance()->parsers[name] = parser;
  }
  ~TFNodeRegistrar() = default;
};
}  // namespace lite
}  // namespace mindspore

#endif  // MINDSPORE_LITE_TOOLS_CONVERTER_PARSER_TF_NODE_PARSER_REGISTRY_H


In [ ]:
/**
 * Copyright 2020 Huawei Technologies Co., Ltd
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

#ifndef MINDSPORE_LITE_TOOLS_CONVERTER_PARSER_TF_NODE_PARSER_H
#define MINDSPORE_LITE_TOOLS_CONVERTER_PARSER_TF_NODE_PARSER_H

#include <string>
#include <map>
#include "tf_util.h"
#include "build/proto/graph.pb.h"
#include "src/ops/primitive_c.h"

namespace mindspore {
namespace lite {
class TFNodeParser {
 public:
  TFNodeParser();

  virtual ~TFNodeParser() {};

  virtual STATUS Parse(const tensorflow::NodeDef *tf_op, const std::unique_ptr<tensorflow::GraphDef> &tf_model,
                       PrimitiveC *primitiveC, int &output_size) {
    return RET_OK;
  }
};
}  // namespace lite
}  // namespace mindspore

#endif  // MINDSPORE_LITE_TOOLS_CONVERTER_PARSER_TF_NODE_PARSER_H


In [ ]:
/**
 * Copyright 2020 Huawei Technologies Co., Ltd
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

#ifndef MINDSPORE_LITE_TOOLS_CONVERTER_PARSER_TF_MODEL_PARSER_H
#define MINDSPORE_LITE_TOOLS_CONVERTER_PARSER_TF_MODEL_PARSER_H

#include <string>
#include <vector>
#include <memory>
#include <map>
#include <unordered_map>
#include "securec/include/securec.h"
#include "tools/common/tensor_util.h"
#include "tools/converter/model_parser.h"
#include "schema/inner/model_generated.h"
#include "build/proto/node_def.pb.h"
#include "build/proto/graph.pb.h"

namespace mindspore {
namespace lite {
class TFModelParser {
 public:
  TFModelParser() = default;
  ~TFModelParser() = default;

  FuncGraphPtr Parse(const std::string &modelFile, const std::string &weightFile, const QuantType &quantType);

 private:
  STATUS ConvertConstTensor(const tensorflow::NodeDef *op, ParameterPtr parameter);
  STATUS ConvertOutputTensor(const tensorflow::NodeDef *op, const CNodePtr &anf_node, int &output_size);
  STATUS ConvertOps();
  STATUS ConvertGraphInputs();
  STATUS ConvertGraphOutputs();

  std::string GetOriginInputName(const tensorflow::NodeDef &node);

  void ClipIdentityAndStopGradient();

  FuncGraphPtr funcGraphPtr;
  std::unique_ptr<tensorflow::GraphDef> tf_graph_def;
  std::map<std::string, const tensorflow::NodeDef *> tf_node_map;
  std::unordered_map<std::string, AnfNodePtr> anf_node_map;
  std::vector<std::string> graph_input_names, graphOutputNames;
};
}  // namespace lite
}  // namespace mindspore

#endif  // MINDSPORE_LITE_TOOLS_CONVERTER_PARSER_TF_MODEL_PARSER_H


In [ ]:
/**
 * Copyright 2020 Huawei Technologies Co., Ltd
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

#ifndef MINDSPORE_LITE_TOOLS_CONVERTER_PARSER_TF_UTIL_H
#define MINDSPORE_LITE_TOOLS_CONVERTER_PARSER_TF_UTIL_H

#include <string>
#include "build/proto/node_def.pb.h"
#include "ir/dtype/type_id.h"
#include "include/errorcode.h"

namespace mindspore {
namespace lite {
class TensorFlowUtils {
 public:
  static bool FindAttrValue(const tensorflow::NodeDef *nodeDef, std::string attr_name,
                            tensorflow::AttrValue &attr_value);

  static bool TfReadProtoFromBinary(const char *filepath, google::protobuf::Message *message);

  static STATUS ParseDataType(const tensorflow::NodeDef *nodeDef, std::string attr_name, TypeId &data_type);
};
}  // namespace lite
}  // namespace mindspore

#endif  // MINDSPORE_LITE_TOOLS_CONVERTER_PARSER_TF_UTIL_H
